In [1]:
%matplotlib inline
from pycocotools.coco import COCO
from pycocoevalcap.eval import COCOEvalCap
from pycocoevalcap.eval_spice import COCOEvalCapSpice

import matplotlib.pyplot as plt
import skimage.io as io
import pylab
pylab.rcParams['figure.figsize'] = (10.0, 8.0)

import json
from json import encoder
encoder.FLOAT_REPR = lambda o: format(o, '.3f')

In [2]:
# set up file names and pathes
dataDir='.'
dataType='val2014'
algName = 'fakecap'
annFile='%s/annotations/captions_%s.json'%(dataDir,dataType)
subtypes=['results', 'evalImgs', 'eval']
[resFile, evalImgsFile, evalFile]= \
['%s/results/captions_%s_%s_%s.json'%(dataDir,dataType,algName,subtype) for subtype in subtypes]

# download Stanford models
! bash get_stanford_models.sh

# download Google word2vec model
! bash get_google_word2vec_model.sh

Found Stanford CoreNLP.
Downloading...
sed: illegal option -- r
usage: sed script [-Ealn] [-i extension] [file ...]
       sed [-Ealn] [-i extension] [-e script] ... [-f script_file] ... [file ...]
--2020-02-23 15:20:46--  https://docs.google.com/uc?export=download&id=0B7XkCwpI5KDYNlNUTTlSS21pQmM
Resolving docs.google.com... 172.217.0.14
Connecting to docs.google.com|172.217.0.14|:443...connected.
HTTP request sent, awaiting response...200 OK
Length: unspecified [text/html]
Saving to: 'STDOUT'

-                       [ <=>                  ]       0  --.-KB/s   in 0s     


Cannot write to '-' (Success).
Code:
--2020-02-23 15:20:48--  https://docs.google.com/uc?export=download&confirm=&id=0B7XkCwpI5KDYNlNUTTlSS21pQmM
Resolving docs.google.com... 172.217.0.14
Connecting to docs.google.com|172.217.0.14|:443...connected.
HTTP request sent, awaiting response...200 OK
Length: unspecified [text/html]
Saving to: 'pycocoevalcap/wmd/data/GoogleNews-vectors-negative300.bin.gz'

pycocoevalcap/wm

In [3]:
import tempfile
preds = json.load(open(resFile, 'r'))
# Create fake predictions
for i in range(1, len(preds), 2):
    preds[i]['image_id'] = preds[i-1]['image_id']
tmp_resFile = tempfile.NamedTemporaryFile('w+')
tmp_resFile.write(json.dumps(preds))
preds[:10]


[{'image_id': 404464,
  'caption': 'black and white photo of a man standing in front of a building'},
 {'image_id': 404464,
  'caption': 'group of people are on the side of a snowy field'},
 {'image_id': 565778, 'caption': 'train traveling down a train station'},
 {'image_id': 565778,
  'caption': 'red fire hydrant sitting on a park bench in front of a road'},
 {'image_id': 322226,
  'caption': 'black and white cat is sitting on top of a wooden bench'},
 {'image_id': 322226, 'caption': 'baseball player swinging a bat at a game'},
 {'image_id': 351053, 'caption': 'laptop computer sitting on top of a table'},
 {'image_id': 351053,
  'caption': 'zebra standing on top of a lush green field'},
 {'image_id': 40102,
  'caption': 'group of giraffes standing next to each other in a grassy field'},
 {'image_id': 40102,
  'caption': 'close up of a pile of oranges sitting on a table'}]

In [4]:
# Eval AllSPICE
coco = COCO(annFile)
cocoRes_n = coco.loadRes(tmp_resFile.name)
cocoEvalAllSPICE = COCOEvalCapSpice(coco, cocoRes_n)
cocoEvalAllSPICE.params['image_id'] = cocoRes_n.getImgIds()
cocoEvalAllSPICE.evaluate()
tmp_resFile.close()

loading annotations into memory...
0:00:00.366050
creating index...
index created!
Loading and preparing results...     
DONE (t=0.01s)
creating index...
index created!
tokenization...
setting up scorers...
computing SPICE score...
SPICE: 0.121


In [5]:
# print output evaluation scores
for metric, score in cocoEvalAllSPICE.eval.items():
    print('%s: %.3f'%('All'+metric, score))

AllSPICE: 0.121
